In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [12]:
attendance = pd.read_csv('../CSV_Files/Attendance.csv')
holiday = pd.read_csv('../CSV_Files/holidays.csv')

C:\Users\siriw\AppData\Local\Temp\ipykernel_8716\41594970.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  attendance = pd.read_csv('../CSV_Files/Attendance.csv')


remove holidays <2021

In [13]:
holiday = holiday.rename(columns={'2017-05-07': 'Holiday'})
# Convert 'Holiday' column to datetime format
holiday['Holiday'] = pd.to_datetime(holiday['Holiday'])
# Filter rows based on the condition
holiday = holiday[holiday['Holiday'].dt.year >= 2021]
holiday = holiday.reset_index(drop=True)

In [14]:
# Check if 'date' and 'out_date' columns are equal
is_equal = (attendance['date'] == attendance['out_date'])

if is_equal.all():
    print("Both columns are equal")
else:
    print("Columns are not equal")

# Count the number of unequal values
unequal_count = (~is_equal).sum()
print(f"Number of unequal values: {unequal_count}")


Columns are not equal
Number of unequal values: 10111


add holidays

In [15]:
# Convert the 'date' column in 'attendance' and 'Holiday' column in 'holiday' to datetime format
attendance['date'] = pd.to_datetime(attendance['date'])
# attendance['out_date'] = pd.to_datetime(attendance['out_date'])
holiday['Holiday'] = pd.to_datetime(holiday['Holiday'])

# Merge the 'attendance' and 'holiday' DataFrames based on matching dates
attendance = pd.merge(attendance, holiday, left_on='date', right_on='Holiday', how='left')

# Set the 'kk' column values based on the merge results
attendance['Holiday_attend'] = attendance['Holiday'].apply(lambda x: 1 if pd.notnull(x) else 0)

# Drop the unnecessary columns ('Holiday') if needed
attendance = attendance.drop(columns=['Holiday'])


In [16]:
attendance = attendance.drop(columns=['date'])
attendance = attendance.drop(columns=['out_date'])

Inside the try-except block, the 'IN' and 'OUT' columns are created as before, indicating if the employee arrived before the shift start time and left after the shift end time, respectively.

In [17]:
import pandas as pd

# Assuming you have an attendance DataFrame called 'attendance' with 'in_time', 'out_time', 'Shift_Start', and 'Shift_End' columns

# Convert time columns to datetime format
attendance['in_time'] = pd.to_datetime(attendance['in_time'], format='%H:%M:%S', errors='coerce').dt.time
attendance['out_time'] = pd.to_datetime(attendance['out_time'], format='%H:%M:%S', errors='coerce').dt.time
attendance['Shift_Start'] = pd.to_datetime(attendance['Shift_Start'], format='%H:%M:%S', errors='coerce').dt.time
attendance['Shift_End'] = pd.to_datetime(attendance['Shift_End'], format='%H:%M:%S', errors='coerce').dt.time

try:
    # Create 'IN' column to indicate if arrived before shift start time
    attendance['One_Time_IN'] = (attendance['in_time'] <= attendance['Shift_Start']).astype(int)

    # Create 'OUT' column to indicate if left after shift end time
    attendance['One_Time_OUT'] = (attendance['out_time'] >= attendance['Shift_End']).astype(int)

except pd.errors.ParserError:
    # If a ParserError occurs, remove the row with the invalid time value
    attendance = attendance.drop(735)


attendance = attendance.drop(columns=['in_time', 'out_time'])
# Preview the updated attendance DataFrame
# attendance


Time shift mapper

In [18]:
import pandas as pd
import numpy as np

# Assuming you have a DataFrame called 'attendance' with 'Shift_Start', 'Shift_End', and 'Piras_Code' columns

# Create a new column 'Piras' and generate unique integer codes
attendance['Piras'] = attendance['Shift_Start'].astype(str) + '_' + attendance['Shift_End'].astype(str)
unique_piras = attendance['Piras'].unique()
attendance['Piras'] = attendance['Piras'].replace('00:00:00_00:00:00', np.nan)
attendance['Piras_Code'] = attendance['Piras'].factorize()[0] + 1

# Create a mapping dictionary for NaN values
nan_map = {np.nan: 'No_Piras'}

# Map the index values for rows with NaN 'Piras' values
attendance['Index_Mapping'] = attendance['Piras'].map(nan_map)

# Print each unique Piras_Code value, corresponding pair of Shift_Start and Shift_End values, and the Index_Mapping
for code in attendance['Piras_Code'].unique():
    piras_pair = attendance[attendance['Piras_Code'] == code][['Shift_Start', 'Shift_End']].iloc[0]
    index_mapping = attendance.loc[attendance['Piras_Code'] == code, 'Index_Mapping'].iloc[0]
    print(f"Piras_Code: {code} - {piras_pair['Shift_Start']} - {piras_pair['Shift_End']}")

# remove unwanted columns
attendance = attendance.drop(columns=['Piras', 'Index_Mapping', 'Shift_Start', 'Shift_End'])

# attendance

Piras_Code: 1 - 08:30:00 - 17:00:00
Piras_Code: 2 - 08:00:00 - 17:00:00
Piras_Code: 3 - 08:30:00 - 13:30:00
Piras_Code: 0 - 00:00:00 - 00:00:00
Piras_Code: 4 - 07:00:00 - 06:59:00
Piras_Code: 5 - 08:00:00 - 07:00:00
Piras_Code: 6 - 18:00:00 - 06:00:00
Piras_Code: 7 - 07:30:00 - 18:00:00
Piras_Code: 8 - 06:00:00 - 07:00:00
Piras_Code: 9 - 19:00:00 - 07:00:00
Piras_Code: 10 - 17:00:00 - 22:00:00
Piras_Code: 11 - 17:00:00 - 08:00:00


In [19]:
attendance = attendance.drop(columns=['id'])
attendance.to_csv('attendance_preprocess.csv', index=False)
attendance

,project_code,Employee_No,Hourly_Time,Holiday_attend,One_Time_IN,One_Time_OUT,Piras_Code
0,100.0,1272,8.5,0,1,1,1
1,1.0,1539,8.5,0,1,1,1
2,1.0,1540,9.25,0,1,1,1
3,100.0,1273,10,0,1,1,1
4,1.0,573,0,0,0,1,1
...,...,...,...,...,...,...,...
224052,206.0,2455,5.03,0,1,0,2
224053,206.0,2453,10.73,0,1,1,2
224054,206.0,2406,0.0,0,0,1,0
224055,206.0,2429,0.0,0,0,1,0
